In [1]:
%pylab
import scipy.integrate as integrate

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# solve_ivp functions

# returns vector of dx/dt, dAcx/dt and dAx/dt in that order

def derivs(t, y, nu, xAs):
    x0 = y[0]               # position of hub
    N = int(0.5*(len(y)-1))   # of lattice sites
    Acx = y[1:N+1]          # bound complex on lattice
    Ax = y[-N:]             # free complex on lattice
    
    dxcdt = integrate.trapz(-(x0-xAs)*Acx, x=xAs)  # force equation
    binding = exp(-0.5*(x0-xAs)**2)*Ax 
    burnt = nu*Acx

    dAcxdt = (binding - burnt)   # complex chemistry
    dAxdt = -binding            # free substrate chemistry
    
    return array( [dxcdt] + list(dAcxdt) + list(dAxdt))     # maybe a better way to put them all together

In [3]:
# define event as having gone to x = 20.0
def event(t, y):
    x0 = y[0]
        
    return 20.0-x0

event.direction = 0        
event.terminal = True     # stop solver if event satisfied

In [4]:
# use solve_ivp to find solution from t0=0 up to a max tf= 400, but stop if x hits x=20.0

# parameters - just 2 params in this model
tf = 400.0

a0_n = 9
nu_n = 15

a0s = logspace(-1, 3, a0_n, base=10)
nus = logspace(-1, 1, nu_n, base=10)

# define domain and lattice
Lmax = 30.0
Lmin = -10.0
dx = 0.1
Nx = int((Lmax-Lmin)/dx) +1
xAs = linspace(Lmin, Lmax, Nx)

print(a0s,nus)

[1.00000000e-01 3.16227766e-01 1.00000000e+00 3.16227766e+00
 1.00000000e+01 3.16227766e+01 1.00000000e+02 3.16227766e+02
 1.00000000e+03] [ 0.1         0.13894955  0.19306977  0.26826958  0.37275937  0.51794747
  0.71968567  1.          1.38949549  1.93069773  2.6826958   3.72759372
  5.17947468  7.19685673 10.        ]


In [5]:

#I.C.

vs = []  # record final speed and burn rate
burnt = []
for a0 in a0s:
    
    A0x = zeros(Nx)
    A0x[xAs > 0.0] = a0
    # A0x = a0*ones(Nx) + 0.1*a0*2.0*(0.5-random.random(Nx))

    # iterate over params
    vtmp = []
    btmp = []
    for nu in nus:
        print(a0, nu)

        x0 = 0.0
        Ax = 1.0*A0x
        Acx = zeros(Nx)

        # initial conditions for all the equations, y0
        y0 = array([x0] + list(Acx) + list(Ax))

        sol = integrate.solve_ivp(lambda t, y: derivs(t, y, nu, xAs), (0., tf), y0, events=[event], method='BDF')

        speed = derivs(sol.t[-1], sol.y[:,-1], nu, xAs)[0]  #dx/dt
        vtmp.append(speed)
        
        Acxf = sol.y[1:1+Nx,-1]  # final amount of complex
        btot = integrate.trapz(nu*Acxf, x=xAs)    # total rate of burnt at final time 
        btmp.append(btot)
        
    vs.append(vtmp)
    burnt.append(btmp)
vs = array(vs)
burnt = array(burnt)

0.1 0.1
0.1 0.13894954943731375
0.1 0.193069772888325
0.1 0.2682695795279726
0.1 0.372759372031494
0.1 0.517947467923121
0.1 0.7196856730011519
0.1 1.0
0.1 1.3894954943731375
0.1 1.9306977288832496
0.1 2.6826957952797246
0.1 3.72759372031494
0.1 5.17947467923121
0.1 7.196856730011517
0.1 10.0
0.31622776601683794 0.1
0.31622776601683794 0.13894954943731375
0.31622776601683794 0.193069772888325
0.31622776601683794 0.2682695795279726
0.31622776601683794 0.372759372031494
0.31622776601683794 0.517947467923121
0.31622776601683794 0.7196856730011519
0.31622776601683794 1.0
0.31622776601683794 1.3894954943731375
0.31622776601683794 1.9306977288832496
0.31622776601683794 2.6826957952797246
0.31622776601683794 3.72759372031494
0.31622776601683794 5.17947467923121
0.31622776601683794 7.196856730011517
0.31622776601683794 10.0
1.0 0.1
1.0 0.13894954943731375
1.0 0.193069772888325
1.0 0.2682695795279726
1.0 0.372759372031494
1.0 0.517947467923121
1.0 0.7196856730011519
1.0 1.0
1.0 1.38949549437313

In [6]:
factor = 0.32
eff_abs = factor* vs**2/burnt

for i in range(0,a0_n):
    plot(nus, eff_abs[i],'.-', label = "$a_{tot}$:  " + str(round(a0s[i],2)))
title(r"$Steady\ State\ Velocity\ vs. a_{tot}$")
legend()
xlabel(r"$\nu'$")
ylabel("Absolute Efficiency")
xscale('log')
yscale('log')

In [7]:
# plot v vs. nu
for i in range(0,a0_n):
    plot(nus, vs[i], '.-', label = r"$a_{tot}:$  " + str(round(a0s[i],2)))

title(r"$Steady\ State\ Velocity\ vs. \nu'$")
legend()
xlabel(r"$\nu'$")
ylabel("velocity")
xscale('log')

In [9]:
# plot v vs. a0 
for i in range(0,nu_n,2):
    plot(a0s, vs.T[i], '.-', label = r"$\nu':$  " + str(round(nus[i],2)))

title(r"$Steady\ State\ Velocity\ vs. a_{tot}$")
legend()
xlabel(r"$a_{tot}$")
ylabel("velocity")
xscale('log')

In [12]:
#plot burn rate vs. nu
for i in range(0,a0_n):
    plot(nus, burnt[i], '.-', label = r"$a_{tot}:$  " + str(round(a0s[i],2)))

title(r"$Burn\ Rate\ vs. \nu'$")
legend()
xlabel(r"$\nu'$")
ylabel("burn rate")
xscale('log')

In [23]:
#plot burn rate vs. a0
for i in range(0,nu_n-6,2):
    plot(a0s, burnt.T[i], '.-', label = r"$\nu':$  " + str(round(nus[i],2)))

title(r"$Burn\ Rate\ vs. a_{tot}$")
legend()
xlabel(r"$a_{tot}$")
ylabel("burn rate")
xscale('log')

In [12]:
#plot efficiency vs. nu
for i in range(0,a0_n):
    plot(nus, vs[i]**2/burnt[i], '.-', label = r"$a_{tot}:$  " + str(round(a0s[i],2)))

title(r"Relative Efficiency vs. $\nu'$")
legend()
xlabel(r"$\nu'$")
ylabel("efficiency")
xscale('log')

In [10]:
#plot efficiency vs. a0
for i in range(0,nu_n,2):
    plot(a0s, vs.T[i]**2/burnt.T[i], '.-', label = r"$\nu'$:  " + str(round(nus[i],2)))

title(r"Relative Efficiency vs. $a_{tot}$")
legend()
xlabel(r"$a_{tot}$")
ylabel("efficiency")
xscale('log')